In [0]:
import traceback
from pyspark.sql.functions import current_timestamp, input_file_name

# -----------------------------------------
# Function to read data using Auto Loader (incremental)
# -----------------------------------------
def read_incremental_data(path: str, fmt: str):
    try:
        logger.info(f"{JOB_NAME} - Initializing Auto Loader from path: {path}")
        schema_path = f"{SOURCE_PATH}/_schema"

        # Auto Loader Stream
        df = (
            spark.readStream
                 .format("cloudFiles")
                 .option("cloudFiles.format", fmt)
                 .option("cloudFiles.schemaLocation", schema_path)
                 .option("cloudFiles.inferColumnTypes", "true")
                 .option("cloudFiles.schemaEvolutionMode", SCHEMA_EVOLUTION_MODE)
                 .load(path)
                # .withColumn("Ingested_Timestamp", current_timestamp())
                # .withColumn("FileName", col("_metadata.file_path"))
        )

        logger.info(f"{JOB_NAME} - Auto Loader stream read initialized successfully")

        return df

    except Exception as e:
        print(f"{JOB_NAME} - Failed to read from path using Auto Loader: {path}")
        print(traceback.format_exc())
        logger.error(f"{JOB_NAME} - Failed to read from path using Auto Loader: {path}")
        logger.error(traceback.format_exc())
        raise RuntimeError(f"{JOB_NAME} - Auto Loader Read failed") from e
